In [1]:
!unzip Data.zip

Archive:  Data.zip
   creating: Data/
  inflating: Data/original-data.csv  
  inflating: Data/Islamabad-Recoveries.csv  
  inflating: Data/Islamabad-Confirmed-Cases.csv  
  inflating: Data/Islamabad-Deaths.csv  
   creating: Data/Abbottabad/
  inflating: Data/Abbottabad/abbottabad-abbottabad-lockdown-14.csv  
  inflating: Data/Abbottabad/cases_in_regions.csv  
  inflating: Data/Abbottabad/covid_out_deaths.csv  
  inflating: Data/Abbottabad/covid_out_hospitalisations.csv  
  inflating: Data/Abbottabad/covid_out_infections.csv  
  inflating: Data/Abbottabad/covid_out_recoveries.csv  
   creating: Data/Islamabad/
  inflating: Data/Islamabad/cases_in_regions.csv  
  inflating: Data/Islamabad/covid_out_deaths.csv  
  inflating: Data/Islamabad/covid_out_hospitalisations.csv  
  inflating: Data/Islamabad/covid_out_infections.csv  
  inflating: Data/Islamabad/covid_out_recoveries.csv  
  inflating: Data/Islamabad/islamabad-smart-lockdown-27.csv  


In [2]:
import pandas as pd
import plotly.graph_objects as go
from datetime import date
from plotly.subplots import make_subplots

In [3]:
def filter_data_by_resample(mask):
    return original_data.loc[
        (original_data['Result'] == mask[0]) & (original_data['PatientResample'] == mask[1])]

def filter_data_by_status(mask):
    return original_data.loc[(original_data['Result'] == mask[0]) & (original_data['Status'] == mask[1])]


def filter_deaths_data():
    filter_data = original_data.loc[original_data['Status'] == "Expired"]
    return filter_data.groupby('Date of Arrival').agg({'Date of Arrival': 'count'}).rename(
        columns={"Date of Arrival": "Total Cases"}).rename_axis("Day").reset_index()


def filter_recoveries_data():
    filter_data = original_data.loc[original_data['Status'].isin(["Recovered","Discharge"])]
    return filter_data.groupby('Date of Arrival').agg({'Date of Arrival': 'count'}).rename(
        columns={"Date of Arrival": "Total Cases"}).rename_axis("Day").reset_index()


def filter_hospitalization_data():
    return original_data.groupby('Date of Arrival').agg({'Date of Arrival': 'count'}).rename(
        columns={"Date of Arrival": "Total Cases"}).rename_axis("Day").reset_index()

def prepare_final_comparison_data(start_day,data, end_day):
    sim_cases = []
    [sim_cases.append(data.count(i)) for i in range(start_day, end_day+start_day+1)]
    return sim_cases

def fill_missing(data,startDay,endDay,key='Day'):
  idx = pd.date_range(startDay,endDay)
  data = data.set_index(key)
  data.index = pd.DatetimeIndex(data.index)
  data = data.reindex(idx, fill_value=0)
  return data

def convert_to_daily_cases(data,key):
  data[key]=data[key] -data[key].shift(1)
  return data.dropna()

def compute_rolling_average(data,window_size):
  i = 0
  moving_averages = []
  while i < len(data) - window_size + 1:
    this_window = data[i : i + window_size]
    window_average = sum(this_window) / window_size
    moving_averages.append(window_average)
    i += 1
  return moving_averages

def make_graph(title,org_data,sim_data,window_size=None,key='Total Cases'):
  original_data = org_data[key].to_list()
  sim_data = prepare_final_comparison_data(comparison_start_day,sim_data, comparison_data_length)
  fig = go.Figure()
  if window_size == None:    
    fig.add_trace(go.Scatter(x=org_data.index, y=original_data,
                            mode='lines+markers',
                            name='Original Data', line=dict(color='orange')))
    fig.add_trace(go.Scatter(x=org_data.index, y=sim_data,
                            mode='lines+markers',
                            name='Simulation Data', line=dict(color='purple')))
  else:
    original_data_roll = compute_rolling_average(original_data,window_size)
    sim_data_roll = compute_rolling_average(sim_data,window_size)
    fig.add_trace(go.Scatter(x=org_data.index[:-window_size+1], y=original_data_roll,
                            mode='lines+markers',
                            name='Original Data', line=dict(color='orange')))
    fig.add_trace(go.Scatter(x=org_data.index[:-window_size+1], y=sim_data_roll,
                            mode='lines+markers',
                            name='Simulation Data', line=dict(color='purple')))
  fig.update_layout(
      title=title,
      xaxis_title="Date",
      yaxis_title="Count",
      font=dict(
          family="Courier New, monospace",
          size=12,
          color="RebeccaPurple"
      )
  )
  fig.show()


def make_graph_combined(org_data,simulation_data,window_size=None,key='Total Cases'):
  labels = ['recoveries','deaths','hospitalization']
  fig = make_subplots(rows=3, cols=1, start_cell="top-left")
  colors = [('orange','purple'),('green','black'),('blue','red')]
  for i in range(3):
    color = colors[i]
    original_data = org_data[i][key].to_list()
    sim_data = prepare_final_comparison_data(comparison_start_day,simulation_data[i], comparison_data_length)
    if window_size == None:    
      x_data = [i for i in range(len(original_data))]
      fig.add_trace(go.Scatter(x=x_data, y=original_data,
                              mode='lines+markers',
                              name='Original '+labels[i]+' Data', line=dict(color=color[0])),row=i+1,col=1,)
      fig.add_trace(go.Scatter(x=x_data, y=sim_data,
                              mode='lines+markers',
                              name='Simulation '+labels[i]+' Data', line=dict(color=color[1])),row=i+1,col=1)
    else:
      original_data_roll = compute_rolling_average(original_data,window_size)
      sim_data_roll = compute_rolling_average(sim_data,window_size)
      x_data = [i for i in range(len(original_data_roll))]
      fig.add_trace(go.Scatter(x=x_data, y=original_data_roll,
                              mode='lines+markers',
                              name='Original '+labels[i]+' Data', line=dict(color=color[0])),row=i+1,col=1)
      fig.add_trace(go.Scatter(x=x_data, y=sim_data_roll,
                              mode='lines+markers',
                              name='Simulation '+labels[i]+' Data', line=dict(color=color[1])),row=i+1,col=1)
  fig.update_layout(
      font=dict(
          family="Courier New, monospace",
          size=12,
          color="RebeccaPurple"
      ),height=800
  )
  fig.update_layout(legend=dict(
    orientation="h",
    yanchor="top",
    y=-0.2,
    xanchor="center",
    x=0.5
))
  fig.update_xaxes(title_text="Days",row=3, col=1)
  fig.update_yaxes(title_text="Population",row=2, col=1)
  fig.show()

## Abbottabad


In [4]:
original_data = pd.read_csv('Data/original-data.csv')
original_data['Date of Arrival'] = pd.to_datetime(original_data['Date of Arrival'])
# original_data = original_data.iloc[:, :-5]


hospitalization_data = pd.read_csv('Data/Abbottabad/covid_out_hospitalisations.csv', names=['Day', 'Lat', 'Lot', 'Age'])
hospitalization_list = hospitalization_data['Day'].to_list()

deaths_data = pd.read_csv('Data/Abbottabad/covid_out_deaths.csv', names=['Day', 'Lat', 'Lot', 'Place'])
deaths_list = deaths_data['Day'].to_list()

recoveries_data = pd.read_csv('Data/Abbottabad/covid_out_recoveries.csv', names=['Day', 'Lat', 'Lot', 'Place'])
recoveries_data = recoveries_data.loc[(recoveries_data['Place']=='hospital')]
recoveries_list = recoveries_data['Day'].to_list()


# hospitalization_data = pd.read_csv('covid_out_hospitalisations.csv', names=['Day', 'Lat', 'Lot', 'Age'])
# hospitalization_list = hospitalization_data['Day'].to_list()

# deaths_data = pd.read_csv('covid_out_deaths.csv', names=['Day', 'Lat', 'Lot', 'Place'])
# deaths_list = deaths_data['Day'].to_list()

# recoveries_data = pd.read_csv('covid_out_recoveries.csv', names=['Day', 'Lat', 'Lot', 'Place'])
# recoveries_list = recoveries_data['Day'].to_list()

In [5]:
SIMULATION_START_DATE = '3-1-2020'
ORIGINAL_DATA_FIRST_DATE = '3-11-2020'
ORIGINAL_DATA_LAST_DATE = '12-13-2020'

sd = SIMULATION_START_DATE.split('-')
ofd = ORIGINAL_DATA_FIRST_DATE.split('-')
old = ORIGINAL_DATA_LAST_DATE.split('-')

simulation_start = date(int(sd[2]),int(sd[0]),int(sd[1]))
original_first_date = date(int(ofd[2]),int(ofd[0]),int(ofd[1]))
original_last_date = date(int(old[2]),int(old[0]),int(old[1]))

comparison_start_day  = (original_first_date - simulation_start).days
comparison_data_length = (original_last_date - original_first_date).days
sim_comparison_length = (original_last_date - simulation_start).days
print(comparison_start_day)
print(comparison_data_length)
print(sim_comparison_length)

10
277
287


In [6]:
original_data = original_data.loc[(original_data['Result'].isin(["Positive","Suspected","Awaiting"]))]

hospitalization_comparison_data = filter_hospitalization_data()

deaths_comparison_data = filter_deaths_data()

recoveries_comparison_data = filter_recoveries_data()

In [7]:
hospitalization_comparison_data_series = fill_missing(hospitalization_comparison_data,original_first_date,original_last_date)
deaths_comparison_data_series = fill_missing(deaths_comparison_data,original_first_date,original_last_date)
recoveries_comparison_data_series = fill_missing(recoveries_comparison_data,original_first_date,original_last_date)

In [8]:
recoveries_comparison_data_series.to_csv("original_recoveries.csv")
deaths_comparison_data_series.to_csv("original_deaths.csv")
hospitalization_comparison_data_series.to_csv("original_hospitalizations.csv")

In [9]:
df = pd.read_csv('Data/Abbottabad/abbottabad-abbottabad-lockdown-14.csv', delimiter=',')

fig = make_subplots(
    rows=3, cols=2,
    specs=[[{}, {}],[{}, {}],[{"colspan": 2}, None]])

fig.add_trace(go.Scatter(x=df['#time'], y=df['susceptible'],
                    mode='lines+markers',
                    name='susceptible',  line=dict(color='orange')),
              row=1, col=1)

fig.add_trace(go.Scatter(x=df['#time'], y=df['exposed'],
                    mode='lines+markers',
                    name='exposed',  line=dict(color='purple')),
              row=1, col=2)

fig.add_trace(go.Scatter(x=df['#time'], y=df['infectious'],
                    mode='lines+markers',
                    name='infectious',  line=dict(color='red')),
              row=3, col=1)

fig.add_trace(go.Scatter(x=df['#time'], y=df['recovered'],
                    mode='lines+markers',
                    name='recovered', line=dict(color='green')),
              row=2, col=1)

fig.add_trace(go.Scatter(x=df['#time'], y=df['dead'],
                    mode='lines+markers',
                    name='dead', line=dict(color='black')),
              row=2, col=2)

# Update xaxis properties
# fig.update_xaxes(title_text="Days", row=1, col=1)
# fig.update_xaxes(title_text="Days",row=1, col=2)
# fig.update_xaxes(title_text="Days",row=2, col=1)
# fig.update_xaxes(title_text="Days",row=2, col=2)
fig.update_xaxes(title_text="Days",row=3, col=1)

# Update yaxis properties
# fig.update_yaxes(title_text="Population", row=1, col=1)
# fig.update_yaxes(title_text="Population", row=1, col=2)
fig.update_yaxes(title_text="Population", row=2, col=1)
# fig.update_yaxes(title_text="Population", row=2, col=2)
# fig.update_yaxes(title_text="Population", row=3, col=1)
fig.show()

In [10]:
make_graph('Recoveries Comparison',recoveries_comparison_data_series,recoveries_list)

In [11]:
make_graph('Recoveries Comparison with Rolling Average (Window = 5)',recoveries_comparison_data_series,recoveries_list,5)

In [12]:
make_graph('Deaths Comparison',deaths_comparison_data_series,deaths_list)

In [13]:
make_graph('Deaths Comparison with Rolling Average (Window = 5)',deaths_comparison_data_series,deaths_list,5)

In [14]:
make_graph('Hospitalization Comparison',hospitalization_comparison_data_series,hospitalization_list)

In [15]:
make_graph('Hospitalization Comparison with Rolling Average (Window = 5)',hospitalization_comparison_data_series,hospitalization_list,5)

In [16]:
org_data = [recoveries_comparison_data_series,deaths_comparison_data_series,hospitalization_comparison_data_series]
sim_data = [recoveries_list,deaths_list,hospitalization_list]
make_graph_combined(org_data,sim_data,window_size=5)

## Islamabad


In [17]:
original_cummulative_cases = pd.read_csv('Data/Islamabad-Confirmed-Cases.csv')
original_cummulative_cases['Date'] = pd.to_datetime(original_cummulative_cases['Date'])
original_daily_cases = convert_to_daily_cases(original_cummulative_cases,'Cases')

original_cummulative_deaths = pd.read_csv('Data/Islamabad-Deaths.csv')
original_cummulative_deaths['Date'] = pd.to_datetime(original_cummulative_deaths['Date'])
original_daily_deaths = convert_to_daily_cases(original_cummulative_deaths,"Deaths")

original_cummulative_recoveries = pd.read_csv('Data/Islamabad-Recoveries.csv')
original_cummulative_recoveries['Date'] = pd.to_datetime(original_cummulative_recoveries['Date'])
original_daily_recoveries = convert_to_daily_cases(original_cummulative_recoveries,"Recoveries")

infection_data = pd.read_csv('Data/Islamabad/covid_out_infections.csv')
infection_list = infection_data['#time'].to_list()

deaths_data = pd.read_csv('Data/Islamabad/covid_out_deaths.csv', names=['Day', 'Lat', 'Lot', 'Place'])
deaths_list = deaths_data['Day'].to_list()

recoveries_data = pd.read_csv('Data/Islamabad/covid_out_recoveries.csv', names=['Day', 'Lat', 'Lot', 'Place'])
recoveries_list = recoveries_data['Day'].to_list()

In [18]:
SIMULATION_START_DATE = '2-26-2020'
ORIGINAL_DATA_FIRST_DATE = '3-11-2020'
ORIGINAL_DATA_LAST_DATE = '12-31-2020'

sd = SIMULATION_START_DATE.split('-')
ofd = ORIGINAL_DATA_FIRST_DATE.split('-')
old = ORIGINAL_DATA_LAST_DATE.split('-')

simulation_start = date(int(sd[2]),int(sd[0]),int(sd[1]))
original_first_date = date(int(ofd[2]),int(ofd[0]),int(ofd[1]))
original_last_date = date(int(old[2]),int(old[0]),int(old[1]))

comparison_start_day  = (original_first_date - simulation_start).days
comparison_data_length = (original_last_date - original_first_date).days
sim_comparison_length = (original_last_date - simulation_start).days

In [19]:
cases_comparison_data_series = fill_missing(original_daily_cases,original_first_date,original_last_date,'Date')
deaths_comparison_data_series = fill_missing(original_daily_deaths,original_first_date,original_last_date,'Date')
recoveries_comparison_data_series = fill_missing(original_daily_recoveries,original_first_date,original_last_date,'Date')

In [20]:
make_graph('Recoveries Comparison',recoveries_comparison_data_series,recoveries_list,key='Recoveries')

In [21]:
make_graph('Recoveries Comparison with Rolling Average (Window = 5)',recoveries_comparison_data_series,recoveries_list,5,key='Recoveries')

In [22]:
make_graph('Deaths Comparison',deaths_comparison_data_series,deaths_list,key='Deaths')

In [23]:
make_graph('Deaths Comparison with Rolling Average (Window = 5)',deaths_comparison_data_series,deaths_list,5,key='Deaths')

In [24]:
make_graph('Daily Cases Comparison',cases_comparison_data_series,infection_list,key='Cases')

In [25]:
make_graph('Daily Cases Comparison with Rolling Average (Window = 5)',cases_comparison_data_series,infection_list,5,key='Cases')